<a href="https://colab.research.google.com/github/salarbalou/Photothermal_Study_CQDs/blob/main/Photothermal_Study_of_CQDs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import plotly.io as pio
pio.templates.default= 'plotly_dark'
from plotly.subplots import make_subplots

In [3]:
CQD = ['red', 'orange', 'yellow', 'green', 'cyan', 'blue', 'violet', 'white']
Conc = ['10x', '5x', '2.5x', '0.75x'] # make 4 solutions of wach with different concentrations 
dc = np.arange(20, 150, 20) # contorlled by arduino

In [4]:
!wget 'https://raw.githubusercontent.com/salarbalou/Photothermal_Study_CQDs/main/red_0.75_20.txt'

--2021-11-24 16:07:15--  https://raw.githubusercontent.com/salarbalou/Photothermal_Study_CQDs/main/red_0.75_20.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4279 (4.2K) [text/plain]
Saving to: ‘red_0.75_20.txt.1’

red_0.75_20.txt.1   100%[===================>]   4.18K  --.-KB/s    in 0s      

2021-11-24 16:07:15 (54.0 MB/s) - ‘red_0.75_20.txt.1’ saved [4279/4279]



In [81]:
 color = 'red'
 concentration = '0.75'
 dc = '20'

In [82]:
 space = '_'
 baseurl = 'https://raw.githubusercontent.com/salarbalou/Photothermal_Study_CQDs/main/'
 format = '.txt'
 data = np.loadtxt(baseurl+color+space+concentration+space+dc+format, delimiter=' , ')
 data = data[9:]

In [83]:
#for blue 435nm maxpower=4W laser:
m, b = 7.025524475524473, -66.07575757575763
# dc is arduino board pwm pin duty cycle

In [84]:
# *** Steady State Analysis *** Heating rate **** #turning on laser for 120 seconds to measure the heating rate
t = data[:, 0][0:121] #time (s)
T = data[:, 1][0:121] #temperature of the solution (C)
Tw = np.full(t.size, np.min(T))#temperature of the environment (C) (needs to be measured by another K type thermocouple) assumed to be constant at 22 C
R  = np.gradient(T, t) #gradient of T with respect to time
HR = np.max(R) #heating rate
Teq = T[np.where(R <= np.min(R))[0]] #equilibrium temperature (C)
teq = t[np.where(R <= np.min(R))[0]] # time required to reach equilibrium (s)
print(f'Heated up to equilibrium eemperature of {Teq} in {teq} seconds')

Heated up to equilibrium eemperature of [34.18] in [77.] seconds


In [106]:
fig = make_subplots()
fig.add_scatter(x= t, y = T, mode = 'markers+lines', marker_color = color, name = f'{concentration}_{dc}')
fig.update_layout(height  = 800, width = 1000)
fig.update_xaxes(nticks = 25, tickfont = dict(size = 18), title = 'time (seconds)', title_font = dict(size = 18))
fig.update_yaxes(nticks = 25, tickfont = dict(size = 18), title = 'Temperature (˚C)', title_font = dict(size = 18))

In [34]:
# *** Steady State Analysis *** Cooling rate **** #turning off laser for 120 seconds to measure the cooling rate
t = data[:, 0] #time (s)
T = data[:, 1] #temperature of the solution (C)
Tw = np.full(t.size, 22)#temperature of the environment (C) (needs to be measured by another K type thermocouple) assumed to be constant at 22 C
C  = np.gradient(T, t) #gradient of T with respect to time
CR = np.max(C) #heating rate
Teq = T[np.min(np.where(C<0.01))] #equilibrium temperature (C)
teq = t[np.min(np.where(C<0.01))] # time required to reach equilibrium (s)
print(f'Cooled down to equilibrium temperature of {Teq} in {teq} seconds')

Cooled down to equilibrium temperature of 23.47 in 9.0 seconds


In [ ]:
# *** Photothermal conversion efficiency ***

#Iout = #measured using powermeter: f(CCQD, L_laser) (W)
#S_PCR_tube = #surface area of the solution at which laser beam is passing through:[PCR_tube, paper] (m**2)
#S_paper = #surface are of the paper at which laser beam is covering:[PCR_tube, paper]  (m**2)
#m = rho * V #mass of solution (Kg)

Iin = m*dc + b #input power of the laser: f(dc) (W)
h = 1000 #heat transfor coefficient of solution with its surrounding (W/(m**2)*C)

rho  = 1000 #density of solution (kg/m**3)
V = 1e-8 #volume of the solution (m**3)
Cp = 4182 #heat capacity of water (J/Kg*C)
#eta = ((R*(Cp*m)+np.sum((h*S*(T - Tw))))/(Iin - Iout)
#print(f'Photothermal Conversion efficiency of your CQD is {eta}')

In [ ]:
# *** what we need to know of each CQD***
#print(f' {Color} CQD reached Teq of {Teq} at {Iout} mW in {teq} seconds, eta: {eta}')

In [ ]:
# ***Modeling the heat diffusion in PCR-Tube and paper***
nx = 2
x = np.arange(nx)
nt = np.int(teq) #temperature rise is evaluated in {t.size} seconds
Tplot = np.full(T.size, np.min(Tw))

In [ ]:
fig = make_subplots()
fig.add_scatter(x = x, y = Tplot, name = '0 s')
for i in range (nt):
  Tplot[0] = T[i]
  Tplot[-1] = np.min(Tw)
  fig.add_scatter(x = np.arange(nx), y = Tplot, name = f'{i+1} s')
fig.update_layout(height = 800, width = 1100)
fig.show()